Table 4.3.1: Gender of all Sex in treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.3.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Substance Group')
cell.assert_one()
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(DOWN).is_not_blank().is_not_whitespace()
sex = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(sex,'Treatment group',DIRECTLY,ABOVE),
            HDim(Clients,'Clients in treatment',DIRECTLY,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','clients-in-treatment'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'] = new_table['Treatment group'].map(
    lambda x: {
        'Total' : ''
        }.get(x, x))

In [8]:
import urllib.request as request
import csv
import io
import requests
new_table['Treatment group'] = new_table['Treatment group'].str.strip()
url="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/basis-of-treatment.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
new_table = pd.merge(new_table, c, how = 'left', left_on = 'Treatment group', right_on = 'Label')
new_table.columns = ['Basis of treatment' if x=='Notation' else x for x in new_table.columns]
new_table = new_table.drop(['Label', 'Parent Notation','Sort Priority'], axis = 1)
new_table['Basis of treatment'] = new_table['Basis of treatment'].fillna('gender/total') 
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [9]:
new_table

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/male,opiate,Count,102803,clients-in-treatment
1,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/female,opiate,Count,38386,clients-in-treatment
2,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/total,opiate,Count,141189,clients-in-treatment
3,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/male,non-opiate-only,Count,17120,clients-in-treatment
4,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/female,non-opiate-only,Count,6610,clients-in-treatment
5,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/total,non-opiate-only,Count,23730,clients-in-treatment
6,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/male,non-opiate-and-alcohol,Count,20279,clients-in-treatment
7,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/female,non-opiate-and-alcohol,Count,7405,clients-in-treatment
8,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/total,non-opiate-and-alcohol,Count,27684,clients-in-treatment
9,gregorian-interval/2017-04-01T00:00:00/P1Y,gender/male,alcohol-only,Count,45668,clients-in-treatment
